In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

import datetime
import os

In [24]:
IMG_SIZE = 128
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         
    validation_split=0.2      
)

train_generator = train_datagen.flow_from_directory(
    'raw-img',           
    target_size=(IMG_SIZE, IMG_SIZE),     
    batch_size=BATCH_SIZE,            
    class_mode='categorical',      
    subset='training'         
)

validation_generator = train_datagen.flow_from_directory(
    'raw-img',          
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'   
)

In [26]:
def build_model(filtersList, denseUnits, shape, activation):
    
    model = models.Sequential()
    
    for i, filters in enumerate(filtersList):
        if i == 0:
            model.add(layers.Conv2D(filters, (3, 3), activation=activation, input_shape=shape, padding='same'))
        else:
            model.add(layers.Conv2D(filters, (3, 3), activation=activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    
    for dense_units in denseUnits:
        model.add(layers.Dense(dense_units, activation=activation))
    
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model


In [27]:
TrainingModels = []
Repetitions = 5
EpochList = [10, 15, 20, 25]

TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [512, 256, 128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [256, 128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))

TrainingModels.append(build_model([2, 4, 8, 16, 32, 64], [128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16, 32], [128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([2, 4, 8, 16], [128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))


In [ ]:
log_path = './training_log.txt'

if not os.path.exists(log_path):
    with open(log_path, 'w') as log_file:
        log_file.write("Timestamp,Model,Epochs,Repetition,Validation Loss,Validation Accuracy\n")

# Loop through training configurations
for model_index, model in enumerate(TrainingModels):
    for epoch in EpochList:
        for repetition in range(1, Repetitions + 1):
            print(f"Training Model {model_index + 1}, Epochs: {epoch}, Repetition: {repetition}")

            # Train the model
            history = model.fit(
                train_generator,
                validation_data=validation_generator,
                epochs=epoch,
                verbose=1  # Show progress
            )

            # Generate a timestamp
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

            # Evaluate the model
            validation_metrics = model.evaluate(validation_generator, verbose=1)
            val_loss, val_accuracy = validation_metrics[0], validation_metrics[1]

            # Save the model with timestamp
            model_path = f'./models/model_{model_index + 1}_epochs_{epoch}_rep_{repetition}_{timestamp}.keras'
            model.save(model_path)
            print(f"Model saved at {model_path}")

            # Log the results
            with open(log_path, 'a') as log_file:
                log_file.write(f"{timestamp},Model_{model_index + 1},{epoch},{repetition},{val_loss:.4f},{val_accuracy:.4f}\n")

            print(f"Logged results: Loss = {val_loss:.4f}, Accuracy = {val_accuracy:.4f}\n")